# Exploratory Data Analysis with Python

---
---

The Process:
1. [Import dependencies and configure settings](#Step-1:-Import-dependencies-and-configure-settings)
2. [Load the data into python](#Step-2:-Load-the-data-into-python)
3. [Look at the raw data](#Step-3:-Look-at-the-raw-data)
4. [Ask questions and make plots to dive deeper](#Step-4:-Ask-questions-and-make-plots-to-dive-deeper)
5. [Outline a plan of action for arriving at the final product](#Step-5:-Outline-a-plan-of-action-for-arriving-at-the-final-product)

---

## Step 1: Import dependencies and configure settings

### Import packages, modules, objects, or functions into the current notebook
TIP: Avoid 'from PACKAGE import *' syntax as it clutters your namespace with objects you may not be aware of

In [ ]:
# BEST PRACTICE: import built-in packages first
# from PACKAGE import OBJECT lets us bring only what we need into our namespace
from warnings import filterwarnings
# import PACKAGE, brings the all modules into one object named PACKAGE
import re

# BEST PRACTICE: import third-party packages second
# from PACKAGE import OBJECT as ALIAS, renames the object in our namespace
from geopy.geocoders import Nominatim as geopy_Nominatim
from nltk.tokenize import word_tokenize as nltk_word_tokenize
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer as nltk_WordNetLemmatizer
from nltk.tag import pos_tag as nltk_pos_tag
from pandas_profiling import ProfileReport
from wordcloud import (
    # use parenthesis to import multiple objects and even add aliases
    STOPWORDS as wordcloud_STOPWORDS,
    WordCloud
)
import matplotlib.pyplot as plt
import nltk
# aliases can be added when fully importing packages
import pandas as pd
# aliases can also be used when importing an individual module from the package
import plotly.express as px
import plotly.io as pio
import seaborn as sns

# BEST PRACTICE: import custom packages last
# for example: import custom_module as cm

### Configure settings by using functions or methods (always check the documentation)
Third-party packages used in this notebook:
- [GeoPy](https://geopy.readthedocs.io/en/stable/)
- [Matplotlib](https://matplotlib.org/)
- [Natural Language Toolkit](https://www.nltk.org/)
- [Pandas](https://pandas.pydata.org/pandas-docs/version/1.2.4/user_guide/index.html)
- [Pandas Profiling](https://pandas-profiling.ydata.ai/docs/master/index.html)
- [Plotly](https://plotly.com/python/)
- [Seaborn](https://seaborn.pydata.org/)
- [WordCloud](https://amueller.github.io/word_cloud/)

In [ ]:
# set python shell filter out warnings and avoid cluttering outputs
filterwarnings(
    'ignore'
)

In [ ]:
# set custom pandas package options by iterating through key-value pairs in a dictionary
for option, value in { # dictionaries are denoted by curly brackets {} or the dict() function
    'display.max_columns': 50,
    'display.max_colwidth': None,
    'display.max_info_columns': 50,
    'display.max_rows': 20,
    'display.precision': 4
}.items(): # the .items() function of a dictionary lets us iterate through key, value pairs
    # we can call a function on the variable we set for the objects we're iterating over
    # in this case those variables are 'option', and 'value' and they represent
    # the key, value pairs from the dictionary above
    pd.set_option(
        option, # this will be 'display.max_columns' etc..
        value   # this will be 50, None, etc..
    )

[Top](#Exploratory-Data-Analysis-with-Python)

---

## Step 2: Load the data into python

### Use tools that 'open' files and load their contents into objects
NOTE: It is possible to build custom functions to read files, but not advised as there are many python libraries that can do so while providing enhanced performance and additional functionality

In [ ]:
# load each dataset into a pandas DataFrame object
cocoon_pharmacy_df = pd.read_csv(
    # the read_csv function from the pandas package will read the file at this location
    '../data/cocoon_center_pharmacy.csv'
)
data_literacy_df = pd.read_csv(
    # the '..' notation indicates that parsing should begin at the parent folder 
    # of this notebook, so if this notebook is in the 'notebooks/' folder then
    # '..' would map to the root folder of this repository (the parent of 'notebooks/') 
    '../data/data_literacy_questionnaire.csv'
)
data_journey_df = pd.read_csv(
    '../data/data_journey_questionnaire.csv'
)
meeting_cadence_df = pd.read_csv(
    '../data/meeting_cadence_survey.csv'
)

[Top](#Exploratory-Data-Analysis-with-Python)

---

## Step 3: Look at the raw data

### Option 1: Define a custom function to determine what is shown on-screen
This is only advised when looking for something specific in the raw dataset like missing data or if the third-party tools are insufficient

In [ ]:
# create an example function with one mandatory and one optional parameter
def display_with_info(
    dataframe: pd.DataFrame,
    name: str = None
) -> None:
    # print dataframe name if passed
    if name:
        print(
            '=' * len(name),
            end = '\n'
        )
        print(
            name,
            end = '\n' + (
                '=' * len(name)
            ) + '\n\n'
        )
    # display dataframe information
    display(
        dataframe.info()
    )
    # label the next output
    print(
        # create the illusion of a border by using hyphens
        '-' * 14
    )
    print(
        'First Ten Rows',
        # create the illusion of a border by using hyphens
        end = '\n' + (
            '-' * 14
        )
    )
    # display the first ten rows of the dataframe
    display(
        dataframe.head(
            5
        )
    )
    # label the next output
    print(
        # create the illusion of a border by using hyphens
        '\n',
        end = (
            '-' * 13
        ) + '\n'
    )
    print(
        'Last Ten Rows',
        # create the illusion of a border by using hyphens
        end = '\n' + (
            '-' * 13
        )
    )
    # display the last ten rows of the dataframe
    display(
        dataframe.tail(
            5
        )
    )

TIP: Placing the names of objects with data into a 'reference' object can assist when operating on mutiple datasets

In [ ]:
# create a dictionary object referencing dataframes and their names
sample_datasets = {
    'Cocoon Center Pharmacy Items & Reviews': cocoon_pharmacy_df,
    'Data Literacy Questionnaire': data_literacy_df,
    'Data Journey Questionnaire': data_journey_df,
    'Meeting Cadence Survey': meeting_cadence_df
}

In [ ]:
# iterate through (dataframe name: dataframe object) 'key: value' pairs 
for df_name, df_object in sample_datasets.items():
    # display each dataframe's summary information and five (5) rows from both ends
    display_with_info(
        df_object,
        df_name
    )
    # add a horizontal border to separate outputs
    print(
        '\n' + (
            '_' * 79
        ),
        end = '\n\n'
    )

### Option 2: Use third-party tools for displaying data dynamically
TIP: Check the [tool's documentation](https://pandas-profiling.ydata.ai/docs/master/index.html) for additional configurations and usage guides 

In [ ]:
# iterate through sample datasets again but display a 'Profile Report' this time
for df_name, df_object in sample_datasets.items():
    display(
        # display the profile report with the configuration below
        ProfileReport(
            # use the dataframe object 'value' from the dictionary
            df_object,
            # create a title using the dataframe name 'key' from the dictionary
            title = f'Profile of {df_name} DataFrame',
            # set additional display option for the report
            html = {
                'navbar_show': True,
                'style': {
                    'primary_color': '#016ba9',
                    'full_width': True
                }
            },
            # don't show a progress bar while the report is being built
            progress_bar = False
        # convert to iframe to allow html to render in a notebook
        ).to_notebook_iframe()
    )

[Top](#Exploratory-Data-Analysis-with-Python)

---

## Step 4: Ask questions and make plots to dive deeper

NOTE: Additional settings can be added at any point in the notebook, but if they are global it is best practice to place them at the top 

In [ ]:
# set default renderer in plotly to iframe and use online CDN
pio.renderers.default = "iframe_connected"

In [ ]:
# set matplotlib package to render plots within the notebook cells
%matplotlib inline

### How many reviews belong to each brand?

In [ ]:
# initialize a figure object
plt.figure(
    # determine an initial size by passing length and width as parameters
    figsize = (30,8)
)
# initialize a plot on the axis by calling seaborn's 'countplot' function
ax = sns.countplot(
    # use the cocoon pharmacy dataframe as the data for the plot
    data = cocoon_pharmacy_df,
    # use the 'brand' column for the x axis
    x = 'brand',
    color = 'blue'
)
# iterate through each subcomponent (patch) drawn on the axis
for p in ax.patches:
    # store the value of the subcomponent in a variable
    x = p.get_x()
    # store the height it's drawn to within the axis
    height = p.get_height()
    # store the width it's drawn to within the axis
    width = p.get_width()
    # if it is located below the displayed axis on the figure we can skip it
    if pd.isnull(height):
        pass
    else:
        # add the subcomponent's value at its location (label the bars)
        ax.text(
            x + width/2,
            height,
            str(height),
            ha = 'center',
            weight = 'bold',
            fontsize = 18
        )
# rotate the labels on the x-axis and change their font size
plt.xticks(
    rotation = 45,
    fontsize = 18
)
# change the font size for labels on the y-axis
plt.yticks(
    fontsize = 18
)
# add a label to the x-axis
plt.xlabel(
    'Brand',
    fontsize = 25
)
# add a label to the y-axis
plt.ylabel(
    'Review Counts',
    fontsize = 25
)
# add a title to the figure
plt.title(
    'Number of reviews per Brand - Total {}'.format(
        # similar to using f' before a string, using .format replaces the {} in
        # the string above with the contents of the object below
        cocoon_pharmacy_df.shape[0]
    ),
    size = 30
)
# display the figure containing the plot
plt.show()

### How many reviews belong to each product category?

In [ ]:
# initialize a figure object
plt.figure(
    # determine an initial size by passing length and width as parameters
    figsize = (30,8)
)
# initialize a plot on the axis by calling seaborn's (sns) 'countplot' function
ax = sns.countplot(
    # use the cocoon pharmacy dataframe as the data for the plot
    data = cocoon_pharmacy_df,
    # use the 'product_cat' for the x-axis values
    x = 'product_cat',
    color = 'blue'
)
# iterate through each subcomponent (patch) drawn on the axis
for p in ax.patches: 
    # store the value of the subcomponent in a variable
    x = p.get_x()
    # store the height it's drawn to within the axis
    height = p.get_height()
    # store the width it's drawn to within the axis
    width = p.get_width()
    # if it is located below the displayed axis on the figure we can skip it
    if pd.isnull(height):
        pass
    else:   
        # add the subcomponent's value at its location (label the bars)
        ax.text(
            x + width/2,
            height,
            str(height),
            ha = 'center',
            weight = 'bold',
            fontsize = 18
        )
# rotate the labels on the x-axis and change their font size
plt.xticks(
    rotation = 45,
    fontsize = 18
)
# change the font size for labels on the y-axis
plt.yticks(
    fontsize = 18
)
# add a label to the x-axis
plt.xlabel(
    'Product Category', 
    fontsize = 25
)
# add label to the y-axis
plt.ylabel(
    'Review Counts',
    fontsize = 25
)
# add a title to the figure
plt.title(
    'Number of reviews per Product Category',
    fontsize = 30
)
# display the figure containing the plot
plt.show()

### How many reviews are there for each product category per brand?

TIP: Create a new object when performing transformations to maintain an 'original' with the raw data

In [ ]:
# create a new dataframe object to view the forms grouped by category and brand
grouped_count_df = cocoon_pharmacy_df.groupby(
    # pass a list of columns we want to groupby
    [
        'product_cat',
        'brand'
    ]
# slice for the 'form' column and get a count
)['form'].count().reset_index().rename(
    # rename the column to an appropriate label after transforming
    {
        'form':'Review Count'
    },
    # axis = 0 means apply to rows, axis = 1 means apply to columns
    axis = 1
# replace the sliced index (which may be out of order) with a new sequential one
).reset_index()

In [ ]:
# initialize a figure object using plotly express' bar plot function
fig = px.bar(
    # use the dataframe we just created above as the data source for the plot
    grouped_count_df,
    # use product category for the x-axis
    x = 'product_cat',
    # use the new review count column for the y-axis
    y = 'Review Count', 
    # color the subplots differently based on the 'brand' column value
    color = 'brand',
    # use a discrete color sequence made for qualitative values 
    color_discrete_sequence = px.colors.qualitative.Dark24
)
# add a label to the x-axis and rotate the ticks
fig.update_xaxes(
    tickangle = 325,
    title = 'Brand'
)
# add a label to the y-axis
fig.update_yaxes(
    title = 'Review Counts'
)
# add a title to the figure
fig.update_layout(
    # 'title' takes a dictionary argument so multiple properties can be passed
    title = dict(
        # create a text - title, key - value pair to define that property
        text = '<b>Number of reviews per Product Category and Brand</b>',
        # font also takes a dictionary to discern different properties
        font = dict(
            # in this case we only define a size property
            size = 24
        )
    ),
    # dictionaries can be created using the dict() function or {k:v} syntax
    xaxis = {
        # this property tells the plot to sort the xaxis in descending order
        'categoryorder': 'total descending'
    }
)
# display the figure containing the plot
fig.show()

### What is the price variation between brands for each product category?

TIP: Grouping object assignments and function calls into cells should be done with readability and debugging in mind

In [ ]:
# create a new dataframe object to view the unique values in a group of columns
brand_product_prices = cocoon_pharmacy_df[
    # slice the dataframe for the following list of columns
    [
        'brand',
        'product_cat',
        'form',
        'volume',
        'price'
    ]
# drop rows with duplicated entries, keeping the last by default 
].drop_duplicates() # closed parenthesis () tells functions to use default args
# overwrite the contents of the volume column by 'assigning' a modified version
brand_product_prices['volume'] = brand_product_prices['volume'].replace(
    # replace values that say 'not available' with a 'null' object
    {
        'not available': None
    }
)
# create a new column called price per ml by using existing columns
brand_product_prices['price_per_ml'] = brand_product_prices['price'].div(
    # use a vectorized operation to divide the prices by the volumes
    brand_product_prices['volume'].astype(
        # astype function converts the text values in this column to decimals
        'float'
    )
)

In [ ]:
# initialize a figure object using plotly express' box plot function
fig = px.box(
    # use the dataframe we just created above as the data source
    brand_product_prices,
    # use brand for the x-axis
    x = 'brand',
    # use the new price per milliliter column for the y-axis
    y = 'price_per_ml',
    # color the subplots based on the 'product category' column value
    color = 'product_cat',
    # show all points instead of hiding outliers past a certain threshold
    points = 'all'
)
# add a label to the x-axis and rotate the ticks
fig.update_xaxes(
    tickangle = 325,
    title = 'Brand'
)
# add a label to the y-axis
fig.update_yaxes(
    title = 'Price per ml'
)
# add a title to the figure
fig.update_layout(
    title = 'PRICE VARIATION PER BRAND AND PRODUCT CATEGORY'
)
# add text to the title in the figure
fig.update_layout(
    # 'title' parameter takes a dictionary so multiple arguments can be passed
    title = dict(
        # add subtitle
        text = '<b>Price variation per product and brand</b>',
        # 'font' also takes a dictionary but we are just changing one property
        font = dict(
            size = 24
        )
    )
)
# display the figure containing the plot
fig.show()


### Where are customers located?

In [ ]:
# clean up text in the customer locations column by 'redefining' the object
cocoon_pharmacy_df['customer_loc'] = cocoon_pharmacy_df['customer_loc']\
.str.strip(  # remove whitespace from the ends of each string (text object)
).str.lower( # turn all characters into lowercase
).replace(
    # replace vague and unused labels
    {
        'the kingdom':'saudi arabia',
        'good':None,
        'not available':None,
        'ksa':'saudi arabia',
        'the kingdom':'saudi arabia',
        'london o':'london, ontario',
        'cheshire':'cheshire, uk',
        '-':None,
        'sale, cheshire':'sale, cheshire, uk',
        'tilbury, essex':'tilbury, essex, uk',
        'middle east':'ksa middle east',
        'kingston':'kingston, uk',
        'awali':'awali, bahrain',
        'riyadh, s. a.':'riyadh, saudi arabia',
        'north':None,
        'south east':None,
        'memo':None
    }
)
# initialize geolocator object to get coordinates from location names
geolocator = geopy_Nominatim(
    user_agent = "s"
)
# display example output of coordinates based on a given location name
geolocator.geocode("Abu Dhabi")

**NOTE: The two (2) cells below do not need to be run if 'cocoon_pharmacy_location_added.csv' is present in the '/data/' folder**

In [ ]:
# create columns with coordinates for the customer locations
new_cocoon_df = cocoon_pharmacy_df.assign(
    Customer_Location_Latitude_Longitude = cocoon_pharmacy_df['customer_loc'].apply(
        # lamdda denotes an anonymous function where the 'caller' is the 'input'
        # in this case if the value is not null we want to get the 'geocode'
        lambda x: geolocator.geocode(x) if pd.notnull(
            geolocator.geocode(x)
        ) else None # otherwise we leave null values alone
    ),
    # use the column created in the lines above to create the two columns below
    Latitude = cocoon_pharmacy_df['Customer_Location_Latitude_Longitude'].apply(
        lambda x: x.latitude if pd.notnull(x) else None
    ),
    Longitude = cocoon_pharmacy_df['Customer_Location_Latitude_Longitude'].apply(
        lambda x: x.longitude if pd.notnull(x) else None
    )
)
# get set of columns in the dataframe excluding the ones pertaining to coordinates
# a set is a container object which only keeps unique values but does not 
# maintain its contents in the order they were placed 
columns_without_customer_location = set(
    new_cocoon_df.columns
).difference(
    # we can use the difference function of sets to exclude a subset of values
    [
        'Customer_Location_Latitude_Longitude',
        'Latitude',
        'Longitude'
    ]
)
# use slicing to remove  duplicates in the non-coordinate columns
new_cocoon_df = new_cocoon_df.loc[
    # slice the dataframe for columns without location and then drop the duplicates
    new_cocoon_df[columns_without_customer_location].drop_duplicates(
        # keep the first instance of a duplicate
        keep  = 'first'
    # use the index of this 'de-duped' dataframe to slice for the 'first' instance of each value
    ).index,
    # ':' by itself slices for all columns
    :
]

TIP: Save python objects back into files to avoid recomputing time-consuming transformations 

In [ ]:
# store the modified dataframe as a .csv file in the 'data' folder
new_cocoon_df.to_csv(
    '../data/cocoon_pharmacy_location_added.csv'
)

**NOTE: The cell below can be run without the preceeding two (2) if 'cocoon_pharmacy_location_added.csv' is present in the '/data/' folder**

In [ ]:
# overwrite the 'cocoon_pharmacy_df' object with a dataframe of our 'new' file
cocoon_pharmacy_df = pd.read_csv(
    '../data/cocoon_pharmacy_location_added.csv',
    # this dataframe was stored as .csv, with the index in the first position
    index_col = 0
)

In [ ]:
# initialize a figure by using plotly express' geographic scatter plot function
fig = px.scatter_geo(
    # use the 'new' dataframe which has customer location coordinates
    cocoon_pharmacy_df,
    # use the column named 'Latitude' for latitude coordinates
    lat = 'Latitude',
    # use the column named 'Longitude' for Longitude coordinates
    lon = 'Longitude',
    # use customer location text to label the box that appears when hovering
    hover_name = 'customer_loc'
)
# set properties for the map the plot will be drawn on
fig.update_geos(
    # set a resolution for the rendered image
    resolution = 50,
    # show land on the map
    showland = True,
    # make the land light green
    landcolor = "LightGreen",
    # show oseans on the map
    showocean = True, 
    # make the oceans light blue
    oceancolor = "LightBlue"
)
# set the margins of the dynamic figure and add a gray background
fig.update_layout(
    margin = dict(
        l = 5,
        r = 5,
        t = 25,
        b = 5
    ),
    paper_bgcolor = "lightGrey"
)
# set new subplot properties
fig.update_traces(
    marker = dict(
        size = 9,
        color = 'yellow',
        opacity = 0.6
    )
)
# add a title to the figure
fig.update_layout(
    title = 'Customer Locations, Cocoon Pharmacy'
)
# display the figure containing the plot
fig.show()

### What are the most common terms used in reviews of each product category?

In [ ]:
# iterate through list of components to download from the NLTK library
for component in [
    'punkt',
    'stopwords',
    'averaged_perceptron_tagger',
    'wordnet',
    'omw-1.4'
]:
    # use nltk's download function to pull the components
    nltk.download(
        component
    )

NOTE: '\\' is an escape character, when used at the end of a line it extends it to include the contents of the one below (escapes newline)

In [ ]:
# clean up the column containing the text body of the customer reviews
cocoon_pharmacy_df['body_review'] = cocoon_pharmacy_df['body_review']\
.str.lower( # convert all the text in the reviews to lowercase 
).apply( 
    # apply an anonymous function to remove all special characters
    lambda x:  re.sub(
        r'[^\w\s]',
        '',
        x
    )
)

In [ ]:
# create a function to generate a wordcloud for a given dataframe
def make_wordcloud(
    # takes one positional argument of Pandas DataFrame type named 'df'
    df: pd.DataFrame
# returns a wordcloud object
) -> WordCloud:
    # create a string object of all the words in all the reviews in the frame
    all_words = ' '.join( # each review body will be separated by empty space
        # assumes each dataframe has a column named 'body_review'
        df['body_review']
    )
    # create an object of the tokens in all reviews in a dataframe, where a 
    # token is a grouping of alphanumeric characters such as a word or punctuation
    tokenized_words = nltk_word_tokenize(
        # this separates a continuous string of text into a list of tokens
        all_words
    )
    # create set of unique english 'stopwords' (i.e. 'and', 'to', 'of')
    # the set() function and curly brackets without 'key:value' pairs both create a set  
    english_stops = {
        # the asterisk '*' at the beginning unpacks the list's contents so all
        # the items within are passed instead of just the container object
        *nltk_stopwords.words(
            # passing a language generates the list of all stopwords nltk has for it 
            'english'
        )
    }
    # create string of all tokenized words that fit some conditions
    all_tokens = ' '.join( # each token will be separated by empty space
        # by using an expression we can 'generate' the contents of a list
        [
            # iterate through the tokens from the tokenized_words object above
            # by placing them in the temporary variable 'token' if they satisfy conditions
            token for token in tokenized_words if ( 
                # first condition checks tokens are longer than 2 characters
                len(token) > 2 
            ) and (  # and statement means both conditions must be true
                # second condition checks tokens are not a stopword
                token not in english_stops # not found in stopword list
            ) and (  # chaining and statements allows us to enforce multiple
                # third condition checks the token is not an empty string
                pd.notna(token)
            ) and (  # placing conditions in parenthesis is not necessary but improves readability
                # fourth condition checks the token consists only of letters
                token.isalpha()
            )
        ]
    )
    # initialize the WordNetLemmatizer from the 'Natural Language Tool Kit'
    lemmatizer = nltk_WordNetLemmatizer() # closed parenthesis since it's a call not an object
    # use the 'Part of Speech' tagger to add what each word token is
    parts_of_speech = nltk_pos_tag( # adds label to tag as 'noun', 'verb', etc.
        # create new list of tokens from those that passed the conditions above
        # i.e. [('word', 'wordtype'), (.., ..)] with a pair of values for each word
        nltk_word_tokenize(
            all_tokens
        )
    )
    # initialize the final object that will contain the wordcloud text  
    wordcloud_tokens = ''
    # iterate through a sequence of integers of size equal to the number of tokens
    for i in range( # generates a sequence from 0 to the number passed by default
        # len function can return the number of objects in a container object
        len(
            # in this case parts_of_speech is a list of labeled tokens
            parts_of_speech
        )
    ):  
        ''' NOTE: triple quotes can be used to create a multi-line comment, like
        - Objects defined in an indented block are temporary by default since
        they are only present during its execution

        - [] notation at the end of an object lets us access its contents if its 
        a container, although the syntax varies by type
        ''' 
        if "V" in parts_of_speech[i][1][0]:
            ''' NOTE: The line above is best read in reverse order, it states that if
            the list in position 0 (first) of the object at position 1 (second) 
            of the object at position i (which is the value we're iterating) 
            in the parts_of_speech list contains the string 'v', then proceed to 
            the indented block below where we create a temporary object with a 
            text string ('v'). We slice the parts_of_speech list this way because 
            it is in the format [('word', 'wordtype'), (.., ..)]
            '''
            pos = 'v'
        else:
            # otherwise we create the same object but with a value of 'n'
            pos = 'n'
        # use the lemmatizer object to group words with inflections together
        lem = lemmatizer.lemmatize(
            # takes the word itself, which is the first object in each pair in the list
            parts_of_speech[i][0],
            # use the 'pos' object from if/else block above to tell the lemmatizer 
            # what it needs to group each word with (i.e. nouns or verbs), converting 
            # them when possible if they're an adjective, conjuction, etc. 
            pos
        )
        # add the lemmatized token to the output string with a space to separate it
        wordcloud_tokens += (lem) + ' ' # += tells python to add to the object and not overwrite it
    # generate wordcloud and store it in an object while passing some arguments
    wordcloud_from_reviews = WordCloud(
        # a list of stopwords to remove from the output
        stopwords = wordcloud_STOPWORDS,
        # what percentage of the words should be displayed horizontally
        prefer_horizontal = 0.9,
        # the maximum number of words to display
        max_words = 100
    ).generate( # call the generate function from the object being initialized
        # pass the tokens that have been cleaned, vetted, and lemmatized
        wordcloud_tokens
    )
    # return the wordcloud object when the function is called
    return wordcloud_from_reviews

In [ ]:
# initialize a figure with one row and four columns to fit multiple subplots
fig,ax = plt.subplots(
    nrows = 1,
    ncols = 4,
    figsize = (25,10)
)
# initialize an object to track the columns in the figure housing the wordclouds 
column_num = 0
# iterate through a list of the top five product categories in descending order
# and define a temporary object named 'category' to reference its contents during iteration
for category in cocoon_pharmacy_df['product_cat'].value_counts( # counts the occurrence of each value in 'product_cat'
).sort_values(
    # sorts the values by descending order of their frequency
    ascending = False
# a colon inside brackets [:VALUE] lets us slice for all values up to (or from) 
# a position, .index grabs the labels for the rows in the dataframe (in this 
# case the categories themselves), so the line below gets the first five labels
# from the sorted list of category frequencies we created in the lines above
)[:4].index:
    # display the value of the 'category' object we're iterating over
    print(
        category
    )
    # add a wordcloud to the figure at the 'column_num' column in the axis (i.e. 0 = first)  
    ax[column_num].imshow(
        # call the function we defined earlier to create wordclouds from a dataframe
        make_wordcloud(
            # pass a slice of the dataframe with all rows with the category we're iterating through
            cocoon_pharmacy_df[cocoon_pharmacy_df['product_cat'] == category]
        )
    )
    # turn off the ticks to display only the images drawn on the figure
    ax[column_num].axis("off")
    # add a title to each image on the figure
    ax[column_num].set_title(
        # f' syntax lets us replace the contents of a {} with the value of an object
        f'Wordcloud - {category}', # e.g 'Wordcloud - Body Dry Oils' if category = 'Body Dry Oils' 
        fontsize = 18
    )
    # increment the object so the following iteration adds a figure to the next column over
    column_num += 1
# remove whitespace around the edges of the image in the figure
fig.tight_layout()
# display the figure containing the plots
plt.show()

[Top](#Exploratory-Data-Analysis-with-Python)

---

## Step 5: Outline a plan of action for arriving at the final product

NOTE:
- Keep the [Value-Information-Analytics](https://catalent.sharepoint.com/sites/EnterpriseDataScience/SitePages/A-Methodical-Approach-to-a-Data-Centric-Problem.aspx) model in mind when determining the viability of an approach
- Not all questions have to be answered at this point, but an understanding of the data and how it can resolve the business problem should be clear before proceeding

---
---